In [1]:
import pandas as pd
from sklearn.svm import SVC

In [2]:
train_set = pd.read_csv('data/train.csv')
test_set = pd.read_csv('data/test.csv')

In [3]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [5]:
train_set.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
test_set.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [7]:
title_mapping = {"Mr.": 1, "Miss.": 2, "Mrs.": 3, "Master.": 4}

train_set['Title'] = train_set['Name'].str.extract(' ([A-Za-z]+\.)', expand=False)
train_set['Title'] = train_set['Title'].map(title_mapping).astype("Int64")
train_set['Title'] = train_set['Title'].fillna(5)

test_set['Title'] = test_set['Name'].str.extract(' ([A-Za-z]+\.)', expand=False)
test_set['Title'] = test_set['Title'].map(title_mapping).astype("Int64")
test_set['Title'] = test_set['Title'].fillna(5)

In [8]:
train_set['Sex'] = train_set['Sex'].map({'male': 0, 'female': 1}).astype(int)
test_set['Sex'] = test_set['Sex'].map({'male': 0, 'female': 1}).astype(int)

In [9]:
train_set['Embarked'] = train_set['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype('Int64')
train_set['Embarked'] = train_set['Embarked'].fillna(3)

test_set['Embarked'] = test_set['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype('Int64')
test_set['Embarked'] = test_set['Embarked'].fillna(3)

In [10]:
train_set['Age'] = train_set.Age.round()

In [11]:
X_train = train_set[train_set.Age.notnull()][['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare']]
X_test = train_set[train_set.Age.isnull()][['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare']]
y = train_set.Age.dropna()
age_classifier = SVC()
age_classifier.fit(X_train, y)
prediction = age_classifier.predict(X_test)
agePrediction = pd.DataFrame(data=prediction, index=X_test.index.values, columns=['Age'])
train_set = train_set.combine_first(agePrediction)

In [12]:
test_set['Age'] = test_set.Age.round()

In [13]:
X_train = test_set[test_set.Age.notnull()][['Sex', 'Title', 'Pclass']]
X_test = test_set[test_set.Age.isnull()][['Sex', 'Title', 'Pclass']]
y = test_set.Age.dropna()
age_classifier = SVC()
age_classifier.fit(X_train, y)
prediction = age_classifier.predict(X_test)
agePrediction = pd.DataFrame(data=prediction, index=X_test.index.values, columns=['Age'])
test_set = test_set.combine_first(agePrediction)

In [14]:
y

0      34.0
1      47.0
2      62.0
3      27.0
4      22.0
       ... 
409     3.0
411    37.0
412    28.0
414    39.0
415    38.0
Name: Age, Length: 332, dtype: float64

In [34]:
i = 0
sets = [['Sex'], ['Age'], ['Sex', 'Title'], ['Sex', 'Title', 'Embarked'], ['Sex', 'Title', 'Embarked', 'Pclass']]
names = [1, 2, 3, 4, 5]
for clasifications in sets:
    X = train_set[clasifications]
    x_test = test_set[clasifications]
    Y = train_set['Survived']
    svc = SVC()
    svc.fit(X, Y)
    Y_pred = svc.predict(x_test)
    acc_svc = round(svc.score(X, Y) * 100, 2)
    submission = pd.DataFrame({
        "PassengerId": test_set["PassengerId"],
        "Survived": Y_pred
    })
    file = "Submissions/Submission {}.csv".format(names[i])
    print("El Score de", file, "es de", acc_svc)
    submission.to_csv(file, index=False)
    i = i + 1

El Score de Submissions/Submission 1.csv es de 78.68
El Score de Submissions/Submission 2.csv es de 63.64
El Score de Submissions/Submission 3.csv es de 79.35
El Score de Submissions/Submission 4.csv es de 79.8
El Score de Submissions/Submission 5.csv es de 82.49
